# Pretraining

In [ ]:
%pip install torch transformers[torch] datasets ipywidgets trl

I am downloading a very small subset of the Wikipeda dataset just for demonstration purposes

In [126]:
from datasets import load_dataset

wiki_data = load_dataset(
    "wikimedia/wikipedia",   
    "20231101.en", 
    split="train[:1000]"
)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [127]:
print(wiki_data['text'][0][:1000])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).

Humans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement f

Let's split the data into train and test

In [2]:
wiki_data = wiki_data.train_test_split(test_size=0.2)

In [10]:
wiki_data

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 200
    })
})

I am going to train a model from scratch I am going to use the Mistral architecture as base. I will use the same tokenize to move from text data to the input index data

In [3]:
from transformers import AutoTokenizer

base_model_id = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

The padding token was missing

In [5]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

Tokenizing the data is easy

In [4]:
outputs = tokenizer(
    wiki_data['train']['text'][0:10],
)

In [4]:
max_length = 512

def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        max_length=max_length, 
        padding='max_length', # longuest 
        return_tensors="pt", 
        add_special_tokens=True
    )

tokenized_datasets = wiki_data.map(
    tokenize_function, 
    batched=True, 
    remove_columns=['id', 'url', 'title', 'text']
)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [32]:
tokenizer.padding_side

'left'

In [36]:
tokenizer.pad_token_id

2

In [5]:
from transformers import MistralForCausalLM, MistralConfig
config = MistralConfig()

In [39]:
config

MistralConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [6]:
config = MistralConfig(
    hidden_size=768,
    sliding_window=768,
    intermediate_size=3072,
    max_position_embeddings=max_length,
    num_attention_heads=16,  
    num_hidden_layers=4,
)

In [7]:
model = MistralForCausalLM(config)

In [9]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 768)
    (layers): ModuleList(
      (0-3): 4 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=384, bias=False)
          (v_proj): Linear(in_features=768, out_features=384, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=768, out_features=3072, bias=False)
          (up_proj): Linear(in_features=768, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((768,), eps=1e-06)
        (post_attention_layernorm): MistralRMSNorm((768,), eps=1e-06)
      )
    )
    (norm)

In [8]:
model_size = sum(t.numel() for t in model.parameters())
model_size

84548352

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [11]:
out = data_collator([
    tokenized_datasets["train"][i] for i in range(10)
])

In [53]:
out['input_ids'][0]

tensor([    1,   415, 17965,   302,   272,   334,  8722,   440, 28725,   835,
         2651,   390,   272, 17965,   302,   272,  3735,   321,  2557,   442,
          272, 17965,   302,  2499, 28725,   349,   396,  1524, 17634,  7761,
          298,   347,   272,  1080, 21079,   312,   577,   302,   272,  6532,
         3387, 28725,   690,   349,  5397,   390,   264, 12602,  8118, 28725,
         6823,   297,  7972,  5014, 28725,   395,   396, 16252,  1999,  5682,
        16006,  1987,   272, 25856,  7278, 28723,  6586,   298,   272,  5737,
          302,  1529,   350,   381, 28725,   272, 17965, 10932,   272,   989,
         7253,  2401,  1074,   302,   272, 11819, 11618,  1339, 28723,  6586,
          298,   272,  1450, 26870,  5737,   302,   650,  2152,  6420, 28725,
          378,   835, 10932, 20486, 28742, 28713, 16439,   304,   264,  2513,
          302,   676,  1520, 28723,    13,    13,  1014,   287, 26896,  2708,
         1016,  1002,   369, 10870,   624,   879,  1024,   272, 

In [54]:
out['labels'][0]

tensor([    1,   415, 17965,   302,   272,   334,  8722,   440, 28725,   835,
         2651,   390,   272, 17965,   302,   272,  3735,   321,  2557,   442,
          272, 17965,   302,  2499, 28725,   349,   396,  1524, 17634,  7761,
          298,   347,   272,  1080, 21079,   312,   577,   302,   272,  6532,
         3387, 28725,   690,   349,  5397,   390,   264, 12602,  8118, 28725,
         6823,   297,  7972,  5014, 28725,   395,   396, 16252,  1999,  5682,
        16006,  1987,   272, 25856,  7278, 28723,  6586,   298,   272,  5737,
          302,  1529,   350,   381, 28725,   272, 17965, 10932,   272,   989,
         7253,  2401,  1074,   302,   272, 11819, 11618,  1339, 28723,  6586,
          298,   272,  1450, 26870,  5737,   302,   650,  2152,  6420, 28725,
          378,   835, 10932, 20486, 28742, 28713, 16439,   304,   264,  2513,
          302,   676,  1520, 28723,    13,    13,  1014,   287, 26896,  2708,
         1016,  1002,   369, 10870,   624,   879,  1024,   272, 

In [9]:
from huggingface_hub import notebook_login

notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="mistral-pretraining",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none", 
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

  0%|          | 0/800 [00:00<?, ?it/s]

{'loss': 7.2598, 'grad_norm': 2.867016553878784, 'learning_rate': 1.8750000000000002e-05, 'epoch': 0.62}
{'train_runtime': 47.2311, 'train_samples_per_second': 16.938, 'train_steps_per_second': 16.938, 'train_loss': 7.013702392578125, 'epoch': 1.0}


TrainOutput(global_step=800, training_loss=7.013702392578125, metrics={'train_runtime': 47.2311, 'train_samples_per_second': 16.938, 'train_steps_per_second': 16.938, 'total_flos': 147388052275200.0, 'train_loss': 7.013702392578125, 'epoch': 1.0})

In [12]:
model.device

device(type='mps', index=0)

In [11]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/338M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-pretraining/commit/42c53fd5a2cc2606719c7dc08db5b7b4d64e8d41', commit_message='End of training', commit_description='', oid='42c53fd5a2cc2606719c7dc08db5b7b4d64e8d41', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
from transformers import pipeline

model_id = "damienbenveniste/mistral-pretraining"
pipe = pipeline("text-generation", model=model_id)
txt = "How are you?"
pipe(txt, num_return_sequences=1)

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/338M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'How are you? (1111111) 11111'}]

In [15]:
txt = "generated_text"
pipe(txt)

[{'generated_text': 'generated_text (; 198198198) was a '}]

# Supervised Learning Fine-tuning

In [16]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")

In [20]:
dataset['instruction'][0]

'Give three tips for staying healthy.'

In [17]:
out = dataset[0]['text']
print(out)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "damienbenveniste/mistral-pretraining"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [23]:
model.config.pad_token_id

In [25]:
from trl import DataCollatorForCompletionOnlyLM

response_template = "### Response:"
response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`

data_collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)


{'input_ids': tensor([[    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13,  3195,   460,
           272,  1712,  6258,  9304, 28804,    13,    13, 27332, 12107, 28747,
            13,  1014,  1712,  6258,  9304,   460,  2760, 28725,  5045, 28725,
           304,  9684, 28723]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
            13,  1014,  1

In [27]:
print(dataset['text'][1])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### Response:
The three primary colors are red, blue, and yellow.


In [28]:
tokenized = tokenizer(
    dataset['text'][1], 
)

data_collator([tokenized])

{'input_ids': tensor([[    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13,  3195,   460,
           272,  1712,  6258,  9304, 28804,    13,    13, 27332, 12107, 28747,
            13,  1014,  1712,  6258,  9304,   460,  2760, 28725,  5045, 28725,
           304,  9684, 28723]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
            13,  1014,  1

In [34]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    output_dir="mistral-supervised",
    dataset_text_field="text",
    max_seq_length=512,
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none", 
)

# args = TrainingArguments(
#     output_dir="mistral-supervised",
#     num_train_epochs=1,
#     push_to_hub=True,
#     report_to="none", 
# )

trainer = SFTTrainer(
    model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=dataset,
    data_collator=data_collator,
    # dataset_text_field='text',  
    # max_seq_length=512, 
)

trainer.train()

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


  0%|          | 0/125 [00:00<?, ?it/s]

{'train_runtime': 51.8778, 'train_samples_per_second': 19.276, 'train_steps_per_second': 2.41, 'train_loss': 6.7959267578125, 'epoch': 1.0}


TrainOutput(global_step=125, training_loss=6.7959267578125, metrics={'train_runtime': 51.8778, 'train_samples_per_second': 19.276, 'train_steps_per_second': 2.41, 'total_flos': 75815608061952.0, 'train_loss': 6.7959267578125, 'epoch': 1.0})

In [35]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-supervised/commit/1bc70e67403782b7c354ee506c08ce3daf6f2760', commit_message='End of training', commit_description='', oid='1bc70e67403782b7c354ee506c08ce3daf6f2760', pr_url=None, pr_revision=None, pr_num=None)

# RLHF

In [36]:
dataset = load_dataset("Anthropic/hh-rlhf", split='train[:1000]')

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

In [40]:
dataset['chosen'][0]

"\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it."

In [117]:
print(dataset[1]['rejected'])



Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

Human: yes they did

Assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

Human: you cant read

Assistant: there’s a lot of stuff humans don’t know


In [41]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        tokenized_chosen = tokenizer(chosen)
        tokenized_rejected = tokenizer(rejected)

        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

tokenized_data = dataset.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [42]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = 'damienbenveniste/mistral-supervised'

reward_model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/338M [00:00<?, ?B/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at damienbenveniste/mistral-supervised and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [44]:
reward_model.config.pad_token_id = tokenizer.pad_token_id

In [45]:
from trl import RewardTrainer, RewardConfig

reward_config = RewardConfig(
    output_dir="mistral-reward",
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none",
)

trainer = RewardTrainer(
    model=reward_model,
    tokenizer=tokenizer,
    args=reward_config,
    train_dataset=tokenized_data,
)

trainer.train()

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/trl/trainer/reward_trainer.py:175: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/trl/trainer/reward_trainer.py:192: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


  0%|          | 0/125 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'train_runtime': 264.7644, 'train_samples_per_second': 3.777, 'train_steps_per_second': 0.472, 'train_loss': 0.691390380859375, 'epoch': 1.0}


TrainOutput(global_step=125, training_loss=0.691390380859375, metrics={'train_runtime': 264.7644, 'train_samples_per_second': 3.777, 'train_steps_per_second': 0.472, 'total_flos': 0.0, 'train_loss': 0.691390380859375, 'epoch': 1.0})

In [46]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-reward/commit/31eb9311b9728a6b27c0a5e4f09afa668dabcf58', commit_message='End of training', commit_description='', oid='31eb9311b9728a6b27c0a5e4f09afa668dabcf58', pr_url=None, pr_revision=None, pr_num=None)

# PPO Training 

In [47]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[-1000:]")

In [122]:
dataset['text'][0]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a list of 5 tasks a virtual assistant can help with\n\n### Response:\n1. Taking notes and creating To-do lists \n2. Setting and managing reminders \n3. Searching the web and collecting relevant data \n4. Scheduling and organizing events \n5. Sending and responding to emails'

In [49]:
from trl import AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer

model_id = 'damienbenveniste/mistral-supervised'

ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [52]:
ppo_model

AutoModelForCausalLMWithValueHead(
  (pretrained_model): MistralForCausalLM(
    (model): MistralModel(
      (embed_tokens): Embedding(32000, 768)
      (layers): ModuleList(
        (0-3): 4 x MistralDecoderLayer(
          (self_attn): MistralSdpaAttention(
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (k_proj): Linear(in_features=768, out_features=384, bias=False)
            (v_proj): Linear(in_features=768, out_features=384, bias=False)
            (o_proj): Linear(in_features=768, out_features=768, bias=False)
            (rotary_emb): MistralRotaryEmbedding()
          )
          (mlp): MistralMLP(
            (gate_proj): Linear(in_features=768, out_features=3072, bias=False)
            (up_proj): Linear(in_features=768, out_features=3072, bias=False)
            (down_proj): Linear(in_features=3072, out_features=768, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): MistralRMSNorm((768,), eps=1e-06)
 

In [67]:
print(dataset['text'][1].split('### Response')[0].strip())

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given a story, (add/edit/compare/remove) an element from it.

### Input:
Once upon a time there was a little girl who loved to read books.


In [94]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(
        sample["text"].split('### Response')[0].strip(), 
    )
    sample["prompt"] = sample["text"].split('### Response')[0].strip()
    return sample

tokenized_dataset = dataset.map(tokenize, batched=False)
tokenized_dataset.set_format(type="torch")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [73]:
tokenized_dataset[0]

{'instruction': 'Create a list of 5 tasks a virtual assistant can help with',
 'input': '',
 'output': '1. Taking notes and creating To-do lists \n2. Setting and managing reminders \n3. Searching the web and collecting relevant data \n4. Scheduling and organizing events \n5. Sending and responding to emails',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a list of 5 tasks a virtual assistant can help with\n\n### Response:\n1. Taking notes and creating To-do lists \n2. Setting and managing reminders \n3. Searching the web and collecting relevant data \n4. Scheduling and organizing events \n5. Sending and responding to emails',
 'input_ids': tensor([    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13,  3998,   264,
          1274,   

In [76]:
def collator(data):    
    return dict((key, [d[key] for d in data]) for key in data[0])

In [86]:
tokenized_dataset[0]

{'instruction': 'Create a list of 5 tasks a virtual assistant can help with',
 'input': '',
 'output': '1. Taking notes and creating To-do lists \n2. Setting and managing reminders \n3. Searching the web and collecting relevant data \n4. Scheduling and organizing events \n5. Sending and responding to emails',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a list of 5 tasks a virtual assistant can help with\n\n### Response:\n1. Taking notes and creating To-do lists \n2. Setting and managing reminders \n3. Searching the web and collecting relevant data \n4. Scheduling and organizing events \n5. Sending and responding to emails',
 'input_ids': tensor([    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13,  3998,   264,
          1274,   

In [80]:
collated = collator(tokenized_dataset)

In [100]:
from trl import PPOConfig, PPOTrainer
from transformers import pipeline

ppo_config = PPOConfig(
    remove_unused_columns=False,
    mini_batch_size=2,
    batch_size=2,
)

ppo_trainer = PPOTrainer(
    model=ppo_model,
    config=ppo_config,
    dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=collator,
)

reward_pipeline = pipeline(model='damienbenveniste/mistral-reward')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [101]:
batch = next(iter(ppo_trainer.dataloader))

In [120]:
batch['input_ids']

[tensor([    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13,  3998,   264,
           908,   369,  4347,  1712,  5287,   304,  5723,   272,  9932,   302,
           706, 28723], device='mps:0'),
 tensor([    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
         12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
         28723,    13,    13, 27332,  3133,  3112, 28747,    13, 28777,   495,
           264,  6817, 23094,   302,   272,  3340,   302,   272,  7865, 28723],
        device='mps:0')]

In [102]:
query_tensors = batch['input_ids']

In [103]:
response_tensors = ppo_trainer.generate(
    query_tensors, 
    pad_token_id=tokenizer.eos_token_id,
    return_prompt=False,
    min_length=-1,
    top_k=0.0,
    top_p=1.0,
    do_sample=True,
    max_new_tokens=10
)

In [104]:
response_tensors

[tensor([ 7018, 28725,   590, 17169, 28723,  1136,   272,  1802, 10866,   302],
        device='mps:0'),
 tensor([20681,   304,  1760,   302, 10725,   304,   334,   708,  6833, 23621],
        device='mps:0')]

In [105]:
batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

In [106]:
batch['response']

['species, they."). As theune coverage of',
 'Stewart and support of hero and C no brightonial']

In [108]:
def get_text(instruction, response):
    return 'Human: {} \n\n Assistant: {}'.format(instruction, response)

In [109]:
texts = [get_text(q, r) for q, r in zip(batch["instruction"], batch["response"])]

In [110]:
texts

['Human: Write an article to explain why people need to start eating healthy foods \n\n Assistant: species, they."). As theune coverage of',
 'Human: Structure and critique a short story. \n\n Assistant: Stewart and support of hero and C no brightonial']

In [111]:
outputs = reward_pipeline(texts)

In [112]:
outputs

[{'label': 'LABEL_1', 'score': 0.5693690180778503},
 {'label': 'LABEL_1', 'score': 0.5224015712738037}]

In [113]:
import torch
rewards = [torch.tensor(output["score"]) for output in outputs]

In [114]:
rewards

[tensor(0.5694), tensor(0.5224)]

In [115]:
ppo_trainer.step(
    query_tensors, 
    response_tensors, 
    rewards
)  

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


{'objective/kl': 0.0,
 'objective/kl_dist': array([0., 0.], dtype=float32),
 'objective/logprobs': array([[-13.745289  , -13.745289  , -13.745289  , -13.745289  ,
         -13.745289  , -13.745289  , -13.745289  , -13.745289  ,
         -13.745289  ,  -8.362318  , -12.250181  ,  -4.1648045 ,
          -2.984612  , -10.074863  ,  -5.0674953 ,  -9.35855   ,
          -2.5046992 ,  -9.629036  ,  -3.4843338 , -12.896368  ,
         -10.659691  ,  -4.8805017 ,  -3.4744205 , -10.111923  ,
          -4.2580366 ,  -8.370234  ,  -7.884919  ,  -8.820232  ,
          -2.4600694 , -11.063954  ,  -3.8971233 ,  -9.30577   ,
          -4.234071  , -12.908957  , -10.006294  , -11.856867  ,
          -6.6308174 ,  -4.1959524 ,  -9.945444  ,  -4.214423  ,
          -3.462892  ,  -2.0073445 , -12.529651  , -11.689817  ,
         -10.912195  ,  -8.945691  ,  -0.1686746 , -11.486065  ,
          -6.9285603 ,  -9.342104  ,  -3.8965287 ,  -8.444826  ,
          -9.350177  ,  -6.862813  ,  -8.728247  ,  -2.95

In [118]:
epochs = 1
for epoch in range(epochs):
    for batch in ppo_trainer.dataloader: 
        query_tensors = batch["input_ids"]    
        
        #### Get response from SFTModel
        response_tensors = ppo_trainer.generate(
            query_tensors, 
            pad_token_id=tokenizer.eos_token_id,
            return_prompt=False,
            min_length=-1,
            top_k=0.0,
            top_p=1.0,
            do_sample=True,
            max_new_tokens=10
        )
        batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    
        #### Compute reward score
        texts = [get_text(q, r) for q, r in zip(batch["instruction"], batch["response"])]
        outputs = reward_pipeline(texts)
        rewards = [torch.tensor(output["score"]) for output in outputs]
    
        #### Run PPO step
        ppo_trainer.step(
            query_tensors, 
            response_tensors, 
            rewards
        ) 
        # break    

In [119]:
ppo_trainer.push_to_hub('mistral-ppo')

model.safetensors:   0%|          | 0.00/338M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-ppo/commit/8fff520d9de6604bd43567fac102a3e5c33c04f3', commit_message='Push model using huggingface_hub.', commit_description='', oid='8fff520d9de6604bd43567fac102a3e5c33c04f3', pr_url=None, pr_revision=None, pr_num=None)